In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR

In [2]:
train_trans = T.Compose([
    T.Resize((224)),
    T.RandomHorizontalFlip(0.3),
    T.RandomVerticalFlip(0.3),
    T.ToTensor()
])
test_trans = T.Compose([
    T.Resize((224,224)),
    T.ToTensor()
])

In [3]:
train_data = ImageFolder(root='/Users/arponbiswas/Images/Ice_cream_images/train',transform=train_trans)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

In [4]:
train_loader.dataset.classes

['bj', 'breyers', 'hd', 'talenti']

In [6]:
train_data

Dataset ImageFolder
    Number of datapoints: 239
    Root location: /Users/arponbiswas/Images/Ice_cream_images/train
    StandardTransform
Transform: Compose(
               Resize(size=224, interpolation=bilinear, max_size=None, antialias=True)
               RandomHorizontalFlip(p=0.3)
               RandomVerticalFlip(p=0.3)
               ToTensor()
           )

In [7]:
device = 'cuda' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu')
device

'mps'

In [10]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layer1 = nn.Sequential(
            nn.Conv2d(3,9,4),
            nn.BatchNorm2d(9),
            nn.ReLU()
        )
        self.conv_layer2 = nn.Sequential(
            nn.Conv2d(9,27,4),
            nn.BatchNorm2d(27),
            nn.ReLU()
        )
        self.conv_layer3 = nn.Sequential(
            nn.Conv2d(27,81,4),
            nn.BatchNorm2d(81),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv_layer4 = nn.Sequential(
            nn.Conv2d(81,162,4),
            nn.BatchNorm2d(162),
            nn.ReLU()
        )

        self.fc1 = nn.Linear(104*104*162,100)
        self.droup = nn.Dropout(0.2)
        self.fc2 = nn.Linear(100,4)
    def forward(self, x):
        x = self.conv_layer1(x)
        x = self.conv_layer2(x)
        x = self.conv_layer3(x)
        x = self.conv_layer4(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [11]:
model = CNN()
model

CNN(
  (conv_layer1): Sequential(
    (0): Conv2d(3, 9, kernel_size=(4, 4), stride=(1, 1))
    (1): BatchNorm2d(9, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv_layer2): Sequential(
    (0): Conv2d(9, 27, kernel_size=(4, 4), stride=(1, 1))
    (1): BatchNorm2d(27, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv_layer3): Sequential(
    (0): Conv2d(27, 81, kernel_size=(4, 4), stride=(1, 1))
    (1): BatchNorm2d(81, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_layer4): Sequential(
    (0): Conv2d(81, 162, kernel_size=(4, 4), stride=(1, 1))
    (1): BatchNorm2d(162, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (fc1): Linear(in_features=1752192, out_features=100, bias=True)
  (droup): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in

In [8]:
def train(Model, Train_loader, Test_loader, Epochs = 10, Lr = 0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr = Lr)
    #sc_optimizer = StepLR(optimizer,step_size=5,gamma=0.1)
    model = Model.to(device)
    model.train()
    for epoch in range(Epochs):
        epoch_loss = 0
        epoch_corr = 0
        for x,y in Train_loader:
            optimizer.zero_grad()
            x,y = x.to(device), y.to(device)
            out = model(x)
            _, y_pred = torch.max(out,dim=1)
            epoch_corr += (y_pred == y).sum().item()
            loss = criterion(out, y)
            epoch_loss += loss
            loss.backward()
            optimizer.step()
        mean_loss = epoch_loss/len(Train_loader)
        print(f"Epoch : {epoch+1} Epoch Loss : {mean_loss} Epoch Correct : {epoch_corr}/{len(train_data)}")




In [30]:
import torch
import torchvision.transforms as T
from PIL import Image
import os

In [31]:
trans = T.Compose([
    T.Resize((224,224)),
    T.ToTensor()
])

In [32]:
image_paths = []
test_dir = '/Users/arponbiswas/Images/Ice_cream_images/test'
for i in os.listdir(test_dir):
    image_paths.append(test_dir+f"/{i}")

In [33]:
image_paths

['/Users/arponbiswas/Images/Ice_cream_images/test/19_hd.png',
 '/Users/arponbiswas/Images/Ice_cream_images/test/40_talenti.png',
 '/Users/arponbiswas/Images/Ice_cream_images/test/64_hd.png',
 '/Users/arponbiswas/Images/Ice_cream_images/test/30_talenti.png',
 '/Users/arponbiswas/Images/Ice_cream_images/test/1_breyers.png',
 '/Users/arponbiswas/Images/Ice_cream_images/test/27_breyers.png',
 '/Users/arponbiswas/Images/Ice_cream_images/test/16_bj.png',
 '/Users/arponbiswas/Images/Ice_cream_images/test/15_hd.png',
 '/Users/arponbiswas/Images/Ice_cream_images/test/68_hd.png',
 '/Users/arponbiswas/Images/Ice_cream_images/test/57_breyers.png',
 '/Users/arponbiswas/Images/Ice_cream_images/test/40_hd.png',
 '/Users/arponbiswas/Images/Ice_cream_images/test/20_bj.png',
 '/Users/arponbiswas/Images/Ice_cream_images/test/43_bj.png',
 '/Users/arponbiswas/Images/Ice_cream_images/test/23_hd.png',
 '/Users/arponbiswas/Images/Ice_cream_images/test/65_breyers.png',
 '/Users/arponbiswas/Images/Ice_cream_ima

In [34]:
labels = []
for path in os.listdir(test_dir):
    labels.append(path.split('_')[1].split('.')[0])

In [35]:
labels

['hd',
 'talenti',
 'hd',
 'talenti',
 'breyers',
 'breyers',
 'bj',
 'hd',
 'hd',
 'breyers',
 'hd',
 'bj',
 'bj',
 'hd',
 'breyers',
 'hd',
 'bj',
 'bj',
 'hd',
 'breyers',
 'talenti',
 'talenti',
 'breyers',
 'bj',
 'breyers',
 'hd',
 'breyers',
 'talenti',
 'talenti',
 'talenti',
 'talenti',
 'talenti',
 'breyers',
 'breyers',
 'hd',
 'breyers',
 'bj',
 'breyers',
 'hd',
 'breyers',
 'bj',
 'breyers',
 'talenti',
 'bj',
 'hd',
 'breyers',
 'hd',
 'bj',
 'breyers',
 'talenti',
 'bj',
 'hd',
 'hd',
 'bj',
 'bj',
 'hd',
 'hd',
 'bj',
 'breyers',
 'breyers',
 'talenti',
 'talenti',
 'breyers',
 'breyers',
 'breyers',
 'hd',
 'bj',
 'hd',
 'bj',
 'breyers',
 'bj',
 'hd',
 'breyers',
 'talenti',
 'talenti',
 'hd',
 'bj',
 'hd',
 'hd',
 'hd',
 'breyers',
 'hd',
 'breyers',
 'bj',
 'talenti',
 'hd',
 'bj',
 'talenti',
 'bj',
 'hd',
 'bj',
 'talenti',
 'hd',
 'hd',
 'bj',
 'breyers',
 'hd',
 'breyers',
 'bj',
 'hd',
 'bj',
 'talenti',
 'hd',
 'hd',
 'talenti',
 'hd',
 'breyers',
 'breyers',

In [36]:
len(labels)

239

In [37]:
device = 'cuda' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu')
model = torch.jit.load("Ice_cream_image_model_100%.pt",map_location=device)
model.eval()

RecursiveScriptModule(
  original_name=CNN
  (conv_layer1): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=Conv2d)
    (1): RecursiveScriptModule(original_name=BatchNorm2d)
    (2): RecursiveScriptModule(original_name=ReLU)
  )
  (conv_layer2): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=Conv2d)
    (1): RecursiveScriptModule(original_name=BatchNorm2d)
    (2): RecursiveScriptModule(original_name=ReLU)
  )
  (conv_layer3): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=Conv2d)
    (1): RecursiveScriptModule(original_name=BatchNorm2d)
    (2): RecursiveScriptModule(original_name=ReLU)
    (3): RecursiveScriptModule(original_name=MaxPool2d)
  )
  (conv_layer4): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=Conv2d)
    (1): RecursiveScriptModule(original_name=BatchNorm2d)
    (2): Recursi

In [38]:
classes = ['bj', 'breyers', 'hd', 'talenti']

In [40]:
score = 0
for img_path in image_paths:
    img = Image.open(img_path).convert('RGB')
    trans_img = trans(img).unsqueeze(0)
    image = trans_img.to(device)
    with torch.no_grad():
        out = model(image)
        _, pred = torch.max(out, dim=1)
        pred = classes[pred]
        label = labels[image_paths.index(img_path)]
        print(f"Prediction : {pred} Actual : {label}")
        if pred == label:
            score += 1
print(f"Result : {score}/{len(image_paths)} Test Accuracy : {(score/len(image_paths))*100}%")

Prediction : hd Actual : hd
Prediction : talenti Actual : talenti
Prediction : hd Actual : hd
Prediction : talenti Actual : talenti
Prediction : breyers Actual : breyers
Prediction : breyers Actual : breyers
Prediction : bj Actual : bj
Prediction : hd Actual : hd


/opt/miniconda3/lib/python3.12/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Prediction : hd Actual : hd
Prediction : breyers Actual : breyers
Prediction : hd Actual : hd
Prediction : bj Actual : bj
Prediction : bj Actual : bj
Prediction : hd Actual : hd
Prediction : breyers Actual : breyers
Prediction : hd Actual : hd
Prediction : bj Actual : bj
Prediction : bj Actual : bj
Prediction : hd Actual : hd
Prediction : breyers Actual : breyers
Prediction : talenti Actual : talenti
Prediction : talenti Actual : talenti
Prediction : breyers Actual : breyers
Prediction : bj Actual : bj
Prediction : breyers Actual : breyers
Prediction : hd Actual : hd
Prediction : breyers Actual : breyers
Prediction : talenti Actual : talenti
Prediction : talenti Actual : talenti
Prediction : talenti Actual : talenti
Prediction : talenti Actual : talenti
Prediction : talenti Actual : talenti
Prediction : breyers Actual : breyers
Prediction : breyers Actual : breyers
Prediction : hd Actual : hd
Prediction : breyers Actual : breyers
Prediction : bj Actual : bj
Prediction : breyers Actual 